In [1]:
# Dependencies and Setup
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import datetime   # handle date times
import re         # regular expression package

!pip install dtale
#!pip install folium
import folium
from folium.plugins import HeatMap

import dtale # data Frame visualization

Select only the columns we are intersted in and select only accidents that happend in the state of Texas

* ID
* Severity
* Start_Time
* End_Time
* Start_Lat
* Start_Lng
* City
* County
* State
* Zipcode
* Country
* Visibility(mi) 
* Weather_Condition
* Precipitation(in)
* Sunrise_Sunset
* Civil_Twilight


Also create data frame for the metro cities in Texas



In [2]:
# Check if data has been pickled and if so load pickle
if os.path.exists('data/txDF.pkl') :
    txDF=pd.read_pickle('data/txDF.pkl',compression='gzip')
    tx_metro_cities_df = pd.read_pickle('data/tx_metro_cities.pkl',compression='gzip')    
else:
# pickle only the attributes we are interested in, and compress
# pickle is much faster/smaller than csv
    df = pd.read_csv('data/US_Accidents_Dec19.csv',encoding='utf-8')
    columns_of_interest=[
        'ID','Severity','Start_Time','End_Time','Start_Lat', 'Start_Lng', 'City', 
        'County', 'State','Zipcode', 'Country','Visibility(mi)', 
        'Weather_Condition','Precipitation(in)','Sunrise_Sunset','Civil_Twilight'
    ]
    condition=df['State'] =='TX'
    txDF=df[condition][columns_of_interest]
    #create df for selected cities 
    tx_metro_cities =  ['Austin', 'Round Rock', 'Cedar Park',
                 'San Marcos', 'Georgetown', 'Pflugerville',
                 'Hutto', 'Buda', 'Kyle', 'Leander',
                 'Dallas', 'Fort Worth', 'Arlington', 
                 'Plano','Garland', 'Irving', 'McKinney', 
                 'Frisco','Denton','Richardson', 'Allen','El Paso',
                 'Houston', 'The Woodlands', 'Sugar Land',
                 'Baytown', 'Conroe','San Antonio', 'New Braunfels', 
                 'Schertz', 'Seguin']
    condition = txDF['City'].isin(tx_metro_cities)
    tx_metro_cities_df = txDF[condition]    
    txDF.to_pickle("data/txDF.pkl",compression='gzip')
    tx_metro_cities_df.to_pickle("data/tx_metro_cities.pkl",compression='gzip')

FileNotFoundError: [Errno 2] File b'data/US_Accidents_Dec19.csv' does not exist: b'data/US_Accidents_Dec19.csv'

In [6]:
# add_datepart from fastai
# Date Feature Engineering
# will split data colum in to corresponding 'Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear'
# 'Hour', 'Minute'
def add_datepart(df, fldname, drop=True,time=False):
    fld = df[fldname]
    attributes = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear']
    if time: attributes = attributes + ['Hour', 'Minute']
    #, 'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in attributes:
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldname, axis=1, inplace=True)
        
add_datepart(tx_metro_cities_df,'Start_Time',False,True )        

In [7]:
# use name for DOW
def dow(df):
    days=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    return days[df]

tx_metro_cities_df['Start_TimeDayofweek'] =tx_metro_cities_df['Start_TimeDayofweek'].apply(dow)

In [8]:
print(f"Number of Columns : {len(txDF.columns)}")
print(f"Number of Rows : {len(txDF)}")
tx_metro_cities_df.columns

Number of Columns : 17
Number of Rows : 298062


Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'City', 'County', 'State', 'Zipcode', 'Country', 'Visibility(mi)',
       'Weather_Condition', 'Precipitation(in)', 'Sunrise_Sunset',
       'Civil_Twilight', 'Counter', 'Start_TimeYear', 'Start_TimeMonth',
       'Start_TimeWeek', 'Start_TimeDay', 'Start_TimeDayofweek',
       'Start_TimeDayofyear', 'Start_TimeHour', 'Start_TimeMinute',
       'Start_TimeElapsed'],
      dtype='object')

In [9]:
print(f"Number of Columns : {len(tx_metro_cities_df.columns)}")
print(f"Number of Rows : {len(tx_metro_cities_df)}")


Number of Columns : 26
Number of Rows : 265605


In [10]:
# get Fatality Date

In [5]:
# make sure to install these packages before running:
#!pip install gmaps
!pip install sodapy
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
import numpy as np
import requests
import gmaps
import json 
import os
import functools 
from sodapy import Socrata

In [6]:

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.austintexas.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.austintexas.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.

results = client.get("ergh-7g8p", limit=2000)
results_df_2012 =pd.DataFrame.from_records(results)

# Convert to pandas DataFrame
results = client.get("vggi-9ddh", limit=2000)
results_df_2013 =pd.DataFrame.from_records(results)

results = client.get("gm9p-snyb", limit=2000)
results_df_2014=pd.DataFrame.from_records(results)

results = client.get("p658-umsa", limit=2000)
results_df_2015=pd.DataFrame.from_records(results)

results = client.get("tiqb-wv3c", limit=2000)
results_df_2016=pd.DataFrame.from_records(results)


results = client.get("ijds-pcyq", limit=2000)
results_df_2017=pd.DataFrame.from_records(results)

results = client.get("9jd4-zjmx", limit=2000)
results_df_2018 = pd.DataFrame.from_records(results)



In [7]:
results_df_2017

,type,fatal_crash_number,number_of_fatalities,case_number,location,area,date,month,day,hour,...,killed_driver_pass,speeding,ran_red_light_or_stop_sign,dl_status_incident,suspected_impairment,restraint_helmet,type_of_road,failure_to_stop_and_render_aid,x_coord,y_coord
0,Motor Vehicle,1,1,17-0030106,8200 blk N. Lamar Blvd,ED,2017-01-03T00:00:00.000,Jan,Tue,1,...,driver,Y,N,okay,DRIVER,seatbelt worn,high use roadway,N,-97.709153,30.350809
1,Bicycle,2,1,17-0080877,Research Blvd SVRD NB/Riata Trace Pkwy,AD,2017-01-08T00:00:00.000,Jan,Sun,15,...,n/a,N,Y,suspended,NONE,helmet worn,other highway,N,-97.753497,30.426752
2,Motor Vehicle,3,1,17-0071308,E US Hwy 290 WB Svrd/Johnny Morris,CH,2017-01-07T00:00:00.000,Jan,Sat,21,...,other driver,Y,N,no DL,DRIVER,unknown,other highway,Y,-97.623275,30.33123
3,Motor Vehicle,4,1,17-0150225,13500 N US 183 NB,AD,2017-01-15T00:00:00.000,Jan,Sun,2,...,passenger,N,N,no DL,BOTH Drivers,no seatbelt,other highway,N,-97.790548,30.449598
4,Pedestrian,5,1,17-0110340,5900 blk N IH35,ID,2017-01-11T00:00:00.000,Jan,Wed,6,...,n/a,N,N,okay,PED,n/a,IH35,N,-97.706646,30.320762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,Motorcycle,67,1,17-3380841,8300 Research Blvd Svrd,ID,2017-12-04T00:00:00.000,DEC,Mon,13,...,driver,Y,N,suspended,Driver,helmet worn,other highway,n,-97.715069,30.355643
67,Motorcycle,68,1,17-3390056,800 W Cesar Chavez,GE,2017-12-05T00:00:00.000,DEC,Tue,0,...,Driver,Y,N,suspended,Driver,no helmet,local,n,-97.754409,30.266275
68,Motor Vehicle,69,1,17-3410022,3302 Northeast,ID,2017-12-07T00:00:00.000,DEC,Thu,0,...,passenger,Y,N,no DL,DRIVER,no seatbelt,local,n,-97.67093,30.308889
69,Motorcycle,70,1,17-3460912,8400 Research Blvd SB,ID,2017-12-12T00:00:00.000,DEC,Tue,14,...,driver,unk,n,okay,NONE,no helmet,other highway,n,-97.715831,30.356976


In [8]:
#concatenate 2016-2018 fatality data
results_project= pd.concat([results_df_2016, results_df_2017, results_df_2018])
results_project

,type,fatal_crash_number,number_of_fatalities,case_number,location,area,date,month,day,hour,...,ftsra,coord_x,y_coord,killed_driver_pass,dl_status_incident,restraint_helmet,failure_to_stop_and_render_aid,x_coord,case_status,restraint_type
0,Pedestrian,1,1,16-0061621,9700 Blk Lake Creek Pkwy,AD,2016-01-06T00:00:00.000,Jan,Wed,23,...,y,-97.793878,30.475234,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pedestrian,2,1,16-0140992,4800 E. Riverside Dr.,HE,2016-01-14T00:00:00.000,Jan,Thu,14,...,n,-97.717471,30.231659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Motor Vehicle,3,1,16-0221932,6300 blk E William Cannon,FR,2016-01-22T00:00:00.000,Jan,Fri,23,...,n,-97.739337,30.170877,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Pedestrian,4,1,16-0301529,12300 N Lamar,ED,2016-01-30T00:00:00.000,Jan,Sat,20,...,n,-97.679087,30.402741,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Motorcycle,5,1,16-0450263,Burleson Rd / McKinney Falls Pkwy,HE,2016-02-14T00:00:00.000,Feb,Sun,2,...,n,-97.698371,30.191989,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,MOTOR VEHICLE,70,1,18-3331578,Research Blvd SVRD SB / MC Cann Dr,NaN,2018-11-29T00:00:00.000,Nov,Thu,19,...,NaN,NaN,30.37397,driver,ok,NaN,n,-97.728905,open,ok
69,Pedestrian,71,1,18-3350146,900 Springdale Rd,CH,2018-12-01T00:00:00.000,Dec,Sat,1,...,NaN,NaN,30.260667,n/a,unk,NaN,y,-97.698705,open,n/a
70,MOTOR VEHICLE,72,1,18-3381590,Payton Gin Rd / Research Blvd Svrd NB,ID,2018-12-04T00:00:00.000,Dec,Tue,20,...,NaN,NaN,30.365314,driver,ok,NaN,n,-97.717789,open,ok
71,Motorcycle,73,1,18-3551763,10200 N MOPAC Expwy SB,AD,2018-12-21T00:00:00.000,Dec,Fri,20,...,NaN,NaN,30.390881,n/a,ok,NaN,n,-97.734239,open,no helmet


In [9]:
#exploring data
len(results_project)

221

In [10]:
results_project.head()

,type,fatal_crash_number,number_of_fatalities,case_number,location,area,date,month,day,hour,...,ftsra,coord_x,y_coord,killed_driver_pass,dl_status_incident,restraint_helmet,failure_to_stop_and_render_aid,x_coord,case_status,restraint_type
0,Pedestrian,1,1,16-0061621,9700 Blk Lake Creek Pkwy,AD,2016-01-06T00:00:00.000,Jan,Wed,23,...,y,-97.793878,30.475234,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pedestrian,2,1,16-0140992,4800 E. Riverside Dr.,HE,2016-01-14T00:00:00.000,Jan,Thu,14,...,n,-97.717471,30.231659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Motor Vehicle,3,1,16-0221932,6300 blk E William Cannon,FR,2016-01-22T00:00:00.000,Jan,Fri,23,...,n,-97.739337,30.170877,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Pedestrian,4,1,16-0301529,12300 N Lamar,ED,2016-01-30T00:00:00.000,Jan,Sat,20,...,n,-97.679087,30.402741,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Motorcycle,5,1,16-0450263,Burleson Rd / McKinney Falls Pkwy,HE,2016-02-14T00:00:00.000,Feb,Sun,2,...,n,-97.698371,30.191989,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
results_project.dtypes.value_counts()

object    30
dtype: int64

In [12]:
results_project.tail()

,type,fatal_crash_number,number_of_fatalities,case_number,location,area,date,month,day,hour,...,ftsra,coord_x,y_coord,killed_driver_pass,dl_status_incident,restraint_helmet,failure_to_stop_and_render_aid,x_coord,case_status,restraint_type
68,MOTOR VEHICLE,70,1,18-3331578,Research Blvd SVRD SB / MC Cann Dr,NaN,2018-11-29T00:00:00.000,Nov,Thu,19,...,NaN,NaN,30.37397,driver,ok,NaN,n,-97.728905,open,ok
69,Pedestrian,71,1,18-3350146,900 Springdale Rd,CH,2018-12-01T00:00:00.000,Dec,Sat,1,...,NaN,NaN,30.260667,n/a,unk,NaN,y,-97.698705,open,n/a
70,MOTOR VEHICLE,72,1,18-3381590,Payton Gin Rd / Research Blvd Svrd NB,ID,2018-12-04T00:00:00.000,Dec,Tue,20,...,NaN,NaN,30.365314,driver,ok,NaN,n,-97.717789,open,ok
71,Motorcycle,73,1,18-3551763,10200 N MOPAC Expwy SB,AD,2018-12-21T00:00:00.000,Dec,Fri,20,...,NaN,NaN,30.390881,n/a,ok,NaN,n,-97.734239,open,no helmet
72,MOTOR VEHICLE,74,1,18-3640187,S US 183 HWY SB / E BEN White EB to 183 SB Ramp,HE,2018-12-30T00:00:00.000,Dec,Sun,2,...,NaN,NaN,30.222337,passenger,ok,NaN,n,-97.678343,open,no seatbelts


In [13]:
results_project.describe()

,type,fatal_crash_number,number_of_fatalities,case_number,location,area,date,month,day,hour,...,ftsra,coord_x,y_coord,killed_driver_pass,dl_status_incident,restraint_helmet,failure_to_stop_and_render_aid,x_coord,case_status,restraint_type
count,221,221,221,221,221,220,221,221,221,221,...,77,77,221,144,143,71,144,144,73,69
unique,5,77,3,221,221,11,200,15,8,24,...,2,77,216,14,12,9,5,143,2,6
top,Pedestrian,53,1,17-3550122,5800 N IH 35 NB,HE,2016-02-14T00:00:00.000,Nov,Sun,2,...,n,-97.763114,30.465592,driver,okay,n/a,n,-97.795281,Closed,n/a
freq,82,3,214,1,1,34,3,29,42,19,...,69,1,2,47,41,23,71,2,37,29


In [14]:
results_project["type"].value_counts

<bound method IndexOpsMixin.value_counts of 0        Pedestrian
1        Pedestrian
2     Motor Vehicle
3        Pedestrian
4        Motorcycle
          ...      
68    MOTOR VEHICLE
69       Pedestrian
70    MOTOR VEHICLE
71       Motorcycle
72    MOTOR VEHICLE
Name: type, Length: 221, dtype: object>

In [17]:
#check for nans
results_project["".isnull().sum()

type                                0
fatal_crash_number                  0
number_of_fatalities                0
case_number                         0
location                            0
area                                1
date                                0
month                               0
day                                 0
hour                                0
time                                0
related                             0
charge                              0
victim                            144
speeding                            3
ran_red_light_or_stop_sign          1
dl_status                         144
suspected_impairment                0
restraint_or_helmet               144
type_of_road                        0
ftsra                             144
coord_x                           144
y_coord                             0
killed_driver_pass                 77
dl_status_incident                 78
restraint_helmet                  150
failure_to_s